In [4]:
from constants import *
from data_loading import *
from data_cleaning import *
from utils import *
from embeddings import *

import csv, os



#import warnings
#warnings.filterwarnings("ignore", category=DeprecationWarning)


# load training set as DataFrame
train = load_df(TRAIN_NEG, TRAIN_POS, TWEET, LABEL, LABEL_NEG, LABEL_POS)
# set patterns to remove, replace, and replace with
to_remove = "<user>"
to_replace = "[^a-zA-Z#]"
replace_value = " "
# clean training set
train = clean(train, TWEET, CLEAN_TWEET, to_remove, to_replace, replace_value)

In [5]:
train.head()

,tweet,label,clean_tweet
id,,,
0,vinco tresorpack 6 ( difficulty 10 of 10 objec...,-1,vinco tresorpack difficulti object disassembl ...
1,glad i dot have taks tomorrow ! ! #thankful #s...,-1,glad dot tak tomorrow #thank #startho
2,1-3 vs celtics in the regular season = were fu...,-1,vs celtic regular season fuck play playoff
3,<user> i could actually kill that girl i'm so ...,-1,could actual kill girl sorri
4,<user> <user> <user> i find that very hard to ...,-1,find hard believ im afraid


In [6]:
def submission_to_csv(predictions, filename):
    if not os.path.exists(PREDS_FOLDER):
        os.makedirs(PREDS_FOLDER)
    predictions.to_csv(PREDS_FOLDER + filename + ".csv", index_label="Id")

### Bag of Words

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(train['clean_tweet'])

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

train_bow = bow

# splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, train['label'], random_state=42, test_size=0.3)


In [5]:
lreg = LogisticRegression()
lreg.fit(xtrain_bow, ytrain) # training the model

prediction = lreg.predict_proba(xvalid_bow) # predicting on the validation set
prediction_int = prediction[:,1] >= 0.3 # if prediction is greater than or equal to 0.3 than 1 else 0
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int) # calculating f1 score

0.7786465875084468

### TF IDF

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(train['clean_tweet'])

In [7]:
train_tfidf = tfidf

xtrain_tfidf = train_tfidf[ytrain.index]
xvalid_tfidf = train_tfidf[yvalid.index]

lreg.fit(xtrain_tfidf, ytrain)

prediction = lreg.predict_proba(xvalid_tfidf)
prediction_int = prediction[:,1] >= 0.3
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int)

0.7774145944529492

### Fasttext (embedding)

In [4]:
#model = fasttext_model(train, model = "skipgram")
#print(model.words)
# TODO use embedding

### Fasttext (prediction)

In [11]:
def classify_fasttext(train,tweets_col = "clean_tweet", csv_name = "fasttext_clean"):
    best_precision = 0;
    i_best = -1;
    max_iter = 100

    for i in range(0,max_iter):
        train_name, test_name = create_labeled_csv(train, tweets_col = "clean_tweet", split_test = True)
        classifier = fasttext.supervised(train_name, 'model_supervised', label_prefix='__label__')

        result = classifier.test(test_name)
        #print('P@1:', result.precision)
        #print('R@1:', result.recall)
        #print('Number of examples:', result.nexamples)
        if result.precision > best_precision:
            # print('\tprecision: ' + str(best_precision) + ' --> ' + str(result.precision))
            best_precision = result.precision
            i_best = i

            labels = classifier.predict(load_txt(TEST_DATA))
            #flatten the result labels
            labels = [y for x in labels for y in x]
            pred_df= pd.DataFrame({'Prediction': labels})
            pred_df.index.name = 'Id' # rename id column
            pred_df.index += 1 #shift to correspond to sample submission
            submission_to_csv(pred_df, csv_name)
        printOver('\033[1mclassifying:\033[0m '+ str( (i+1)/ max_iter * 100) + '%, best_acc=' + str(best_precision))



#### on "clean_tweet"

In [ ]:
classify_fasttext(train, "clean_tweet", "fasttext_clean")

classifying: 100.0%, best_acc=0.91782705432364192705432364193Best precision:  0.9178270543236419


#### on "tweet"

In [ ]:
classify_fasttext(train, "tweet", "fasttext_noclean")

classifying: 2.0%, best_acc=0.7391065223369416

In [11]:
# TODO fix MemoryError 
#classifier2 = fasttext.load_model('models/word_deps_sg_500d/words500.npy')